In [1]:
import numpy as np
import pandas as pd
import time
import random
import os
import re
import json
import ast
import seaborn as sns
from helper_functions_assignment2 import*

In [2]:
folder_path = "data/inst_competition/"
problem_instances = [file.replace('.txt', '') for file in os.listdir(folder_path) if not file == "test.txt"]

# ALNS

### run for all best configs from SMAC3

In [3]:
# read in the file with the best configurations
best_configs = 'ALNS_smac_results.txt'
with open(best_configs, 'r') as file:
    file_content = file.read()

# Extract configurations using regular expressions
config_pattern = r"Configuration\(values=\{([^}]*)\}\)"
config_values = re.findall(config_pattern, file_content)

# Convert configurations to a list of dictionaries
config_dicts = []
for config_val in config_values:
    config_dict = ast.literal_eval('{' + config_val + '}')
    config_dicts.append(config_dict)

In [4]:
test_results = []
for c in config_dicts:
    print(c)
    for p in problem_instances:
        print(p)
        path = folder_path+p+".txt"
        node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)

        start = time.time()
        plex_assignment = np.random.choice(10, n)
        number_of_phases = round(4000 / c["iterations_per_phase"])
        repair_solution(node_impact_orig, node_degree_orig, plex_assignment, edge_weights, edge_assignment_orig, s)

        
        plex_assignment, edge_assignment, node_impact, node_degree, score, destroy, repair = ALNS(min_weight = c["min_weight"],
                                                                                 reaction_factor = c["reaction_factor"], 
                                                                                 iterations_per_phase = c["iterations_per_phase"], 
                                                                                 number_of_phases = number_of_phases,
                                                                                 node_impact = node_impact_orig, 
                                                                                 node_degree = node_degree_orig, 
                                                                                 edge_assignment = edge_assignment_orig, 
                                                                                 edge_weights = edge_weights, 
                                                                                 plex_assignment = plex_assignment, 
                                                                                 s = s, 
                                                                                 trajectory = False)

        print("instance " + str(p) + ": " + str(destroy) + ", " + str(repair))

        test_results.append({"Instance": p,
                             "min_weight": c["min_weight"], 
                             "reaction_factor": c["reaction_factor"], 
                             "iterations_per_phase": c["iterations_per_phase"], 
                             "number_of_phases": number_of_phases,
                             "edge_assignment": edge_assignment,
                             "baseline": sum(node_impact_orig)/2,
                             "score": score, 
                             "runtime": time.time()-start})

{'iterations_per_phase': 19, 'min_weight': 0.16739073976874352, 'reaction_factor': 0.2578662525396794}
heur049_n_300_m_17695
time_limit
instance heur049_n_300_m_17695: [0.14081559 0.2014398  0.14081559 0.14195152 0.37497751], [0.22306042 0.2342658  0.31961336 0.22306042]
heur050_n_300_m_19207
time_limit
instance heur050_n_300_m_19207: [0.2 0.2 0.2 0.2 0.2], [0.25 0.25 0.25 0.25]
heur051_n_300_m_20122
time_limit
instance heur051_n_300_m_20122: [0.2 0.2 0.2 0.2 0.2], [0.25 0.25 0.25 0.25]
{'iterations_per_phase': 38, 'min_weight': 0.14030814811587333, 'reaction_factor': 0.5839582003559918}
heur049_n_300_m_17695
instance heur049_n_300_m_17695: [0.2 0.2 0.2 0.2 0.2], [0.25 0.25 0.25 0.25]
heur050_n_300_m_19207
time_limit
instance heur050_n_300_m_19207: [0.26693393 0.18326652 0.18326652 0.18326652 0.18326652], [0.23538076 0.23538076 0.23538076 0.29385773]
heur051_n_300_m_20122
time_limit
instance heur051_n_300_m_20122: [0.2 0.2 0.2 0.2 0.2], [0.25 0.25 0.25 0.25]
{'iterations_per_phase': 89

In [5]:
results = pd.DataFrame(test_results)
results["instance_ID"] = results["Instance"].str[4:7]
results

,Instance,min_weight,reaction_factor,iterations_per_phase,number_of_phases,edge_assignment,baseline,score,runtime,instance_ID
0,heur049_n_300_m_17695,0.167391,0.257866,19,211,"[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...",51246.0,29882.0,240.419317,049
1,heur050_n_300_m_19207,0.167391,0.257866,19,211,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",57645.0,40593.0,240.403879,050
2,heur051_n_300_m_20122,0.167391,0.257866,19,211,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...",58898.0,25775.0,240.071846,051
3,heur049_n_300_m_17695,0.140308,0.583958,38,105,"[1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, ...",51658.0,21445.0,240.666223,049
4,heur050_n_300_m_19207,0.140308,0.583958,38,105,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",57934.0,32977.0,240.126187,050
5,heur051_n_300_m_20122,0.140308,0.583958,38,105,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",59008.0,20652.0,240.414251,051
6,heur049_n_300_m_17695,0.067264,0.036799,89,45,"[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, ...",51162.0,21767.0,240.978862,049
7,heur050_n_300_m_19207,0.067264,0.036799,89,45,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",57671.0,38304.0,240.147576,050
8,heur051_n_300_m_20122,0.067264,0.036799,89,45,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",58642.0,16464.0,240.059449,051
9,heur049_n_300_m_17695,0.141922,0.823344,78,51,"[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, ...",51486.0,21436.0,240.056320,049


### get the rows with best results for each instance

In [6]:
results.loc[results.groupby('Instance').score.idxmin()]

,Instance,min_weight,reaction_factor,iterations_per_phase,number_of_phases,edge_assignment,baseline,score,runtime,instance_ID
15,heur049_n_300_m_17695,0.080686,0.582704,37,108,"[1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, ...",51681.0,21027.0,240.123074,049
4,heur050_n_300_m_19207,0.140308,0.583958,38,105,"[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",57934.0,32977.0,240.126187,050
8,heur051_n_300_m_20122,0.067264,0.036799,89,45,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",58642.0,16464.0,240.059449,051


### save best results

In [7]:
for idx in results.groupby('Instance').score.idxmin():
    path = folder_path+ results.loc[idx].Instance+".txt"
    node_impact_orig, node_degree_orig, plex_assignment, edges_n1, edges_n2, edge_weights, edge_assignment_orig, s, n, m = create_problem_instance(path)
    
    write_solution(results.loc[idx].edge_assignment, 
                   results.loc[idx].Instance, 
                   edge_weights, 
                   edges_n1, 
                   edges_n2, 
                   algorithm = "ALNS")